# Hybrid Model for HPE - Estimation of 27 body keypoint positions for each frame of input videos, for further kinematic and inverse dynamic analysis.

**Step 1 (VideoPose3D output) -** This code generates two output folders: *'output_videos_VideoPose3D'* and *'coordinates_VideoPose3D'*. Each folder contains individual subfolders for each subject. The *'output_videos_VideoPose3D'*  folders includes an `.mp4` animation of the predicted movements, while the *'coordinates_VideoPose3D'*  folders contains both `.npz` and `.txt` files that store the coordinates of 17 body keypoints for every frame of the corresponding input video, as predicted by VideoPose3D. Input videos should be placed in a designated folder, with each video filename containing a unique identifying number to distinguish between subjects. (Runtime Mode - *T4 GPU*)

In [ ]:
#Import files from Google drive to Google colab

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Install necessary libraries and imports.
!pip install pillow==9.4.0 idna==3.8

In [ ]:
#Install necessary libraries and imports.

import importlib

#Install ffmpeg
try:
    importlib.import_module('ffmpeg')
    print("ffmpeg is already installed.")
except ImportError:
    print("Installing ffmpeg...")
    !apt-get install ffmpeg -y

#Install Detectron2
try:
    importlib.import_module('detectron2')
    print("detectron2 is already installed.")
except ImportError:
    print("Installing detectron2...")
    !pip install 'git+https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13'

#Install Opencv
try:
    importlib.import_module('cv2')
    print("opencv-python is already installed.")
except ImportError:
    print("Installing opencv-python...")
    !pip install opencv-

#Install torch 2.2
try:
    import torch
    if torch.__version__ == '2.2.0':
        print("torch 2.2 is already installed.")
    else:
        raise ImportError
except ImportError:
    print("Installing torch 2.2...")
    !pip install torch==2.2.0 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118


try:
    importlib.import_module('glob')
    print("glob is already imported.")
except ImportError:
    print("Importing glob...")
    from glob import glob

try:
    importlib.import_module('sys')
    print("sys is already imported.")
except ImportError:
    print("Importing sys...")
    import sys

try:
    importlib.import_module('argparse')
    print("argparse is already imported.")
except ImportError:
    print("Importing argparse...")
    import argparse

#Components of detectron2

import torch, detectron2
try:
    nvcc_output = !nvcc --version
    TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
    CUDA_VERSION = torch.__version__.split("+")[-1]
    print("torch:", TORCH_VERSION, "; cuda:", CUDA_VERSION)
    print("detectron2:", detectron2.__version__)
except Exception as e:
    print("Error:", e)

import json
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import re
%matplotlib inline

# Check if GPU is available
if torch.cuda.is_available():
    GPU = True
    print('GPU available')
else:
    GPU = False
    print('No GPU')

In [ ]:
#Install necessary libraries and imports.
!pip install tqdm==4.66.1 tensorboard==2.15.1 platformdirs==4.1.0 numpy==1.23.5 fonttools==4.46.0 grpcio==1.60.0 google-auth==2.17.3 markdown==3.5.1 huggingface_hub==0.19.4 safetensors==0.4.1 typing-extensions==4.5.0 jinja2==3.1.2 MarkupSafe==2.1.3

In [ ]:
#Change directory to the VideoPose3D folder

main_directory = '/content/drive/MyDrive/VideoPose3D'

#Change the current working directory to the desired directory
os.chdir(main_directory)

#Verify current working directory
current_directory = os.getcwd()

print("Current Working Directory:", current_directory)

In [ ]:
#If this cell fails to run, please try running the cells above again
#Get the path of the folder containing input videos
folder_videos = input("Enter the path of the folder containing the input video files: ")

#Check if the folder exists
if not os.path.exists(folder_videos) or not os.path.isdir(folder_videos):
    print(f"The folder '{folder_videos}' does not exist.")
    exit()

# Current working directory - the directory of the VideoPose3D folder
folder_path = os.getcwd()

# Create 'detections' folder in /content directory
folder_detections = 'detections'
path_detections = os.path.join('/content', folder_detections)
if not os.path.exists(path_detections):
    os.makedirs(path_detections)
    print(f"The folder '{path_detections}' was created.")
else:
    print(f"The folder '{path_detections}' already exists.")

# Create 'output_videos' folder in /content directory
folder_output = 'output_videos_VideoPose3D'
path_output = os.path.join('/content', folder_output)
if not os.path.exists(path_output):
    os.makedirs(path_output)
    print(f"The folder '{path_output}' was created.")
else:
    print(f"The folder '{path_output}' already exists.")

# Create 'coordinates' folder in /content directory
folder_coordinates = 'coordinates_VideoPose3D'
path_coordinates = os.path.join('/content', folder_coordinates)
if not os.path.exists(path_coordinates):
    os.makedirs(path_coordinates)
    print(f"The folder '{path_coordinates}' was created.")
else:
    print(f"The folder '{path_coordinates}' already exists.")

#inferring 2D keypoints with detectron2
path_inference = os.path.join(folder_path, 'inference', 'infer_video_d2.py')
!python "{path_inference}" \
  --cfg COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml \
  --output-dir "{path_detections}" \
  --image-ext mp4 \
  "{folder_videos}"  # directory of the folder with input videos

#Creating a custom dataset
path_data = os.path.join(folder_path, 'data')
%cd "{path_data}"
!python prepare_data_2d_custom.py -i "{path_detections}" -o myvideos

#Video/Animation with estimated 3D coordinates
%cd "{folder_path}"

file_names = []

for filename in os.listdir(folder_videos): #Loop through the filenames in the input folder. Ensure each file has a unique number for identification
    if filename.startswith('.') or filename.endswith('.txt'):  # Skip non-video files.
        continue

    #Extract number from the filename
    number = re.search(r'\d+', filename).group()

    #Name for the output folders of each subject
    output_folder = f'subject_{number}'
    coordinates_folder = f'subject_{number}'

    #Name for the output files of each subject
    output_filename = f'output_{number}.mp4'
    coordinates_filename = f'coordinates_{number}.npy'

    #Creating individual subject folders in the 'output_videos' folder
    path_output_subject = os.path.join(path_output, output_folder)
    if not os.path.exists(path_output_subject):
        os.makedirs(path_output_subject)
        print(f"The folder '{path_output_subject}' was created.")

    # creating individual subject folders in the 'coordinates' folder
    path_coordinates_subject = os.path.join(path_coordinates, coordinates_folder)
    if not os.path.exists(path_coordinates_subject):
        os.makedirs(path_coordinates_subject)
        print(f"The folder '{path_coordinates_subject}' was created.")

    file_names.append(filename)

    command1 = f'python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject {filename} --viz-action custom --viz-camera 0 --viz-video "{os.path.join(folder_videos, filename)}" --viz-output "{os.path.join(path_output_subject, output_filename)}" --viz-size 6'
    command2 = f'python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject {filename} --viz-action custom --viz-camera 0 --viz-video "{os.path.join(folder_videos, filename)}" --viz-export "{os.path.join(path_coordinates_subject, coordinates_filename)}" --viz-size 6'

    print(f'Executing command: {command1}')
    print(f'Executing command: {command2}')

    try:
        os.system(command1)
        os.system(command2)

    except Exception as e:
        print(f'Error occurred: {e}')

#Convert npz file to txt file
for filename in file_names:
  number = re.search(r'\d+', filename).group()

  npz_file = os.path.join(path_coordinates, f'subject_{number}', f'coordinates_{number}.npy')
  txt_output_file = os.path.join(path_coordinates, f'subject_{number}', f'coordinates_{number}.txt')

  try:
        np.set_printoptions(threshold=np.inf)  #To keep all values of the array
        data = np.load(npz_file)
        precision = 8

        with open(txt_output_file, "w") as f:
            for frame_index, frame in enumerate(data, start=1):  #Indicates the number of frames (starts by 1)
                f.write(f"Frame {frame_index}:\n")
                for keypoint_index, keypoint in enumerate(frame, start=1):  #17 keypoints per frame
                    keypoint_str = np.array2string(keypoint, precision=precision, separator=', ', floatmode='fixed')
                    f.write(f"Keypoint {keypoint_index}: {keypoint_str}\n")

        print(f"Converted {npz_file} to {txt_output_file}")
  except Exception as e:
        print(f"Error converting {npz_file}: {e}")


print('Done.')

**Step 2 (MediaPipe Pose output) -** This code generates an output folder (*'coordinates_MediaPipePose'*) containing individual subfolders for each subject. Inside each subject's folder, a `.txt` file is created that stores the coordinates of 33 body keypoints for every frame of the corresponding input video, as predicted by MediaPipe Pose. The input videos must be placed in a designated folder, and each video filename should include a unique number to distinguish between subjects.

In [ ]:
#Installations necessary for MediaPipe Pose
#If the session crashes, please run this cell again to ensure all dependencies are installed.

!pip install --upgrade pip setuptools
!pip install numpy==1.19.3
!pip install -q mediapipe

In [ ]:
import os
import re
import cv2
from google.colab.patches import cv2_imshow
import mediapipe as mp

#Initialize MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

#Function to print the coordinates of the 33 keypoints for each frame (to disable printing the body landmarks for each frame, comment out the call to this function below)
#def write_landmarks_to_csv(landmarks, frame_number, csv_data):
#    print(f"Landmark coordinates for frame {frame_number}:")
#    for idx, landmark in enumerate(landmarks):
#        print(f"{mp_pose.PoseLandmark(idx).name}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z})")
#        csv_data.append([frame_number, mp_pose.PoseLandmark(idx).name, landmark.x, landmark.y, landmark.z])
#    print("\n")

#Specify the directory of the folder with the input files (videos)
folder_videos = input("Enter the path of the folder containing the input video files: ")

# Check if the folder exists
if not os.path.exists(folder_videos) or not os.path.isdir(folder_videos):
    print(f"The folder '{folder_videos}' does not exist.")
    exit()

# Create 'output_videos' folder in /content directory
folder_output = 'coordinates_MediaPipePose'
path_output = os.path.join('/content', folder_output)
if not os.path.exists(path_output):
    os.makedirs(path_output)
    print(f"The folder '{path_output}' was created.")
else:
    print(f"The folder '{path_output}' already exists.")

file_names = []

for filename in os.listdir(folder_videos): #Loop through the filenames in the input folder. Ensure each file has a unique number for identification
    if filename.startswith('.') or filename.endswith('.txt'):  # Skip non-video files.
        continue

    #Extract number from the filename
    number = re.search(r'\d+', filename).group()

    #Create an output folder for each subject (based on the number)
    output_folder = f'subject_{number}'
    path_output_subject = os.path.join(path_output, output_folder)
    if not os.path.exists(path_output_subject):
        os.makedirs(path_output_subject)
        print(f"The folder '{path_output_subject}' was created.")
    else:
        print(f"The folder '{path_output_subject}' already exists.")

    file_names.append(filename)

# Process each video
for filename in file_names:

  file_path = os.path.join(folder_videos, filename)

  # Open the video file
  cap = cv2.VideoCapture(file_path)

  # Extract the number from the filename
  number = re.search(r'\d+', filename).group()

  # Create the output .txt file for a given subject
  output_file_txt = os.path.join(path_output, f'subject_{number}', f'coordinates_{number}.txt')

  frame_number = 0
  csv_data = []

  # Create a list to store the keypoints positions for the entire video
  keypoints_position_list = []

  # Loop through each frame of the video
  while cap.isOpened():
      # Read a frame from the video
      ret, frame = cap.read()
      if not ret:
          break  # Break the loop if no frame read

      # Convert the frame to RGB
      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

      # Process the frame with MediaPipe Pose
      result = pose.process(image)

      if result.pose_world_landmarks:
          keypoints_position_list.append(result.pose_world_landmarks) #Save the keypoints for the frame
          #mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS) # Draw landmarks on the frame (comment the line below if you don't want to display landmarks)
          #write_landmarks_to_csv(result.pose_world_landmarks.landmark, frame_number, csv_data) # Print the keypoint coordinates for the frame (comment the line below to disable printing)


      #Display the frame (comment this line if you don't want to show the frames while processing).
      #cv2_imshow(frame)

      # Save the keypoints of all frames in a txt file
      with open(output_file_txt, "w") as f:
          for frame_index, frame_landmarks in enumerate(keypoints_position_list, start=1): #Indicates the number of frames (starts by 1)
              print(f"Frame {frame_index}:", file=f)
              for keypoint_index, landmark in enumerate(frame_landmarks.landmark, start=1): #33 keypoints per frame
                  print(f"Keypoint {keypoint_index}: [{landmark.x}, {landmark.y}, {landmark.z}]", file=f)

      frame_number += 1

      # Exit if 'q' key pressed
      if cv2.waitKey(1) & 0xFF == ord('q'):
          break


**Step 3 -** Calculation of additional body keypoints - IJ, PX, EM, EL, RS, US, AC. Output: `.txt` files with the coordinates of each keypoint for all the frames of the input videos.

In [ ]:
#function that gets the coordinates of each keypoint
import numpy as np
def get_coordinates(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()

    coordinates_txt = {} #dictionary with the number of frames, and the number of keypoints

    frame_number = None

    for line in lines:
        if line.startswith("Frame"):
            frame_number = int(line.split()[1].strip(':'))  #"split" to separate the different components and "strip" removes the colon, white spaces
            coordinates_txt[frame_number] = {}
        elif line.startswith("Keypoint"):
            parts = line.strip().split()
            keypoint_number = int(parts[1].strip(':'))
            coordinates = [float(coord.strip('[],')) for coord in parts[2:] if coord.strip('[],')] #remove square brackets and commas, convert to float
            coordinates_txt[frame_number][keypoint_number] = coordinates

    return coordinates_txt

In [ ]:
#function that converts global to local coordinates

def glob_2_loc_coords_3D(pointP, origin, basis_P1, basis_P2, basis_P3):
    v1 = basis_P2 - basis_P1
    v2 = basis_P3 - basis_P1

    v1_norm = v1 / np.linalg.norm(v1)
    v2_norm = v2 / np.linalg.norm(v2)

    v3_norm = (np.cross(v1_norm, v2_norm)) / np.linalg.norm(np.cross(v1_norm, v2_norm))

    v4_norm = (np.cross(v1_norm, v3_norm)) / np.linalg.norm(np.cross(v1_norm, v3_norm))

    basis = np.array([[v1_norm[0], v3_norm[0], v4_norm[0]],
                      [v1_norm[1], v3_norm[1], v4_norm[1]],
                      [v1_norm[2], v3_norm[2], v4_norm[2]]])

    return (basis.T).dot(pointP - origin)

In [ ]:
#function that converts local to global coordinates

def loc_2_glob_coords_3D(loc_coords, origin, basis_P1, basis_P2, basis_P3):

    global_coordinates = np.zeros((basis_P1.shape[0], 3))

    if basis_P1.shape[0] == basis_P2.shape[0] == basis_P3.shape[0]:
        v1 = basis_P2 - basis_P1
        v2 = basis_P3 - basis_P1

        v1_norm = v1 / np.linalg.norm(v1)
        v2_norm = v2 / np.linalg.norm(v2)

        v3_norm = (np.cross(v1_norm, v2_norm)) / np.linalg.norm(np.cross(v1_norm, v2_norm))

        v4_norm = (np.cross(v1_norm, v3_norm)) / np.linalg.norm(np.cross(v1_norm, v3_norm))

        basis = np.array([[v1_norm[0], v3_norm[0], v4_norm[0]],
                          [v1_norm[1], v3_norm[1], v4_norm[1]],
                          [v1_norm[2], v3_norm[2], v4_norm[2]]])

        return origin + basis.dot(loc_coords)

    else:
        raise

In [ ]:
#folder with the subfolders with the .txt files of each input video generated by the VideoPose3D algorithm
folder_keypoints = '/content/coordinates_VideoPose3D'

# Use os.walk to traverse through all subdirectories and files
for root, dirs, files in os.walk(folder_keypoints):
    for filename in files:
        if not filename.endswith('.txt'):
            continue  # Skip non-txt files

        keypoints_file_dynamic = os.path.join(root, filename)

        # Check if the file exists and is a file
        if not os.path.isfile(keypoints_file_dynamic):
            print(f"The .txt file '{keypoints_file_dynamic}' does not exist.")
            continue

        # Read the coordinates from the .txt file
        data = get_coordinates(keypoints_file_dynamic)

        # Generate a static trial - first 20 frames of the input video
        output_file = os.path.join(root, f'static_{filename}')  # Save the static trial in the same subfolder as the dynamic trial

        with open(output_file, 'w') as f:
            for frame_number in range(1, 21):
                if frame_number in data:
                    f.write(f"Frame {frame_number}:\n")
                    coords = data[frame_number]
                    for keypoint_number in sorted(coords.keys()):
                        coordinates = coords[keypoint_number]
                        coords_str = ' '.join(map(str, coordinates))
                        f.write(f"Keypoint {keypoint_number}: [{coords_str}]\n")

        print(f"Static trial for {filename} written to: {output_file}")


In [ ]:
folder_keypoints = '/content/coordinates_VideoPose3D'

#calculate IJ and PX for the frames of the static trial
for root, dirs, files in os.walk(folder_keypoints):
    for filename in files:
        if filename.startswith('static') and filename.endswith('.txt'):
            static_file = os.path.join(root, filename)
            number = re.search(r'\d+', filename).group()

            keypoints_data = get_coordinates(static_file)


            local_coordinates_list_IJ = []
            local_coordinates_list_PX = []

            #IJ keypoint
            for frame_number, keypoints in keypoints_data.items():
                spine = np.array(keypoints[8])
                shoulder_right = np.array(keypoints[15])
                shoulder_left = np.array(keypoints[12])
                hip_right = np.array(keypoints[2])
                hip_left = np.array(keypoints[5])
                mid_shoulder = (shoulder_right + shoulder_left) / 2
                mid_hip = (hip_right + hip_left) / 2
                C7_global = np.array(keypoints[9])
                vector1 = shoulder_left - shoulder_right
                vector2 = spine - shoulder_right
                vector1_unit = vector1 / np.linalg.norm(vector1)
                vector2_unit = vector2 / np.linalg.norm(vector2)
                normal_vector = -np.cross(vector1_unit, vector2_unit) #vector perpendicular to the plane formed by the two shoulders and spine (T8), pointing forward
                normal_vector_unit = normal_vector / np.linalg.norm(normal_vector) #transform to unit vector
                normal_vector_1 = normal_vector_unit * ((np.linalg.norm(C7_global - shoulder_right) * 0.1079) / 0.1887) #ratio between the distance from IJ to C7 and the distance from C7 to shoulder
                IJ_global = C7_global + normal_vector_1

                keypoints_data[frame_number][18] = IJ_global

                #convert to local coordinates
                IJ_local = glob_2_loc_coords_3D(IJ_global, mid_hip, mid_hip, spine, hip_right)

                #save local coordinates for all frames of the static test in a list
                local_coordinates_list_IJ.append(IJ_local)

            local_coordinates_array_IJ = np.array(local_coordinates_list_IJ)

            #calculate the average
            IJ_local_final = np.mean(local_coordinates_array_IJ, axis=0)


            #PX keypoint
            for frame_number, keypoints in keypoints_data.items():
                spine = np.array(keypoints[8])
                shoulder_right = np.array(keypoints[15])
                shoulder_left = np.array(keypoints[12])
                hip_right = np.array(keypoints[2])
                hip_left = np.array(keypoints[5])
                mid_shoulder = (shoulder_right + shoulder_left) / 2
                mid_hip = (hip_right + hip_left) / 2
                IJ_global = np.array(keypoints[18])
                C7_global = np.array(keypoints[9])
                theta2 = 36.6823 #angle between vector C7-IJ and vector C7-PX
                theta_rad2 = np.deg2rad(theta2) #change to radians
                vector_sag_1 = IJ_global - C7_global
                vector_sag_2 = spine - C7_global
                vector_sag_1_unit = vector_sag_1 / np.linalg.norm(vector_sag_1)
                vector_sag_2_unit = vector_sag_2 / np.linalg.norm(vector_sag_2)
                vector_perp = np.cross(vector_sag_1_unit, vector_sag_2_unit) #vector perpendicular to the sagittal plane (IJ, C7 and spine(T8))
                vector_perp_unit = vector_perp / np.linalg.norm(vector_perp) #unit vector

                #rotation using Rodrigues' Rotation Formula
                cos_theta2 = np.cos(theta_rad2)
                sin_theta2 = np.sin(theta_rad2)
                omega_x, omega_y, omega_z = vector_perp_unit

                rotation_matrix1 = np.array([[cos_theta2 + omega_x**2 * (1 - cos_theta2), omega_x * omega_y * (1 - cos_theta2) - omega_z * sin_theta2, omega_x * omega_z * (1 - cos_theta2) + omega_y * sin_theta2],
                    [omega_y * omega_x * (1 - cos_theta2) + omega_z * sin_theta2, cos_theta2 + omega_y**2 * (1 - cos_theta2), omega_y * omega_z * (1 - cos_theta2) - omega_x * sin_theta2],
                    [omega_z * omega_x * (1 - cos_theta2) - omega_y * sin_theta2, omega_z * omega_y * (1 - cos_theta2) + omega_x * sin_theta2, cos_theta2 + omega_z**2 * (1 - cos_theta2)]])

                vector_C7_PX = np.dot(rotation_matrix1, vector_sag_1_unit) #C7-IJ vector rotation
                vector_C7_PX_unit = vector_C7_PX / np.linalg.norm(vector_C7_PX) #unit vector
                vector5 = vector_C7_PX_unit * ((np.linalg.norm(C7_global - shoulder_right) * 0.2684) / 0.1887) #ratio between the distance from C7 to PX and the distance from C7 to SJC (C7 to the shoulder)

                PX_global = C7_global + vector5

                keypoints_data[frame_number][19] = PX_global

                #convert to local coordinates
                PX_local = glob_2_loc_coords_3D(PX_global, mid_hip, mid_hip, spine, hip_right)

                #save local coordinates for all frames of the static test in a list
                local_coordinates_list_PX.append(PX_local)

            local_coordinates_array_PX = np.array(local_coordinates_list_PX)

            #calculate the average
            PX_local_final = np.mean(local_coordinates_array_PX, axis=0)

            # Save the calculated keypoints in the static file
            with open(static_file, 'w') as f:
                for frame_number, keypoints in keypoints_data.items():
                    f.write(f"Frame {frame_number}:\n")
                    for keypoint_number, coordinates in keypoints.items():
                        f.write(f"Keypoint {keypoint_number}: {coordinates}\n")

            print(f"New keypoints added to: {static_file}")

            # Find and process the corresponding dynamic file
            for file in files:
                if not file.startswith('static') and file.endswith('.txt'):
                    dynamic_number = re.search(r'\d+', file).group()
                    if number == dynamic_number:
                        dynamic_file = os.path.join(root, file)
                        break

            if dynamic_file:
                keypoints_data = get_coordinates(dynamic_file)

                #use the local coordinates calculated in the static trial, in the dynamic trial
                for frame_number, keypoints in sorted(keypoints_data.items()):
                    spine = np.array(keypoints[8])
                    hip_right = np.array(keypoints[2])
                    hip_left = np.array(keypoints[5])
                    mid_hip = (hip_right + hip_left) / 2

                    keypoints_data[frame_number][18] = loc_2_glob_coords_3D(IJ_local_final, mid_hip, mid_hip, spine, hip_right)
                    keypoints_data[frame_number][19] = loc_2_glob_coords_3D(PX_local_final, mid_hip, mid_hip, spine, hip_right)

                # Save the calculated keypoints in the dynamic file
                with open(dynamic_file, 'w') as f:
                    for frame_number, keypoints in keypoints_data.items():
                        f.write(f"Frame {frame_number}:\n")
                        for keypoint_number, coordinates in keypoints.items():
                            f.write(f"Keypoint {keypoint_number}: {coordinates}\n")

                print(f"New keypoints added to: {dynamic_file}")

print("Processing completed for all static and dynamic trials.")


In [ ]:
folder_keypoints = '/content/coordinates_VideoPose3D'

#folder with the subfolders with the .txt files of each input video generated by the MediaPipe Pose algorithm
folder_keypoints_1 = '/content/coordinates_MediaPipePose'

# Calculate EL, EM, RS, US, AC for the dynamic trial
for root, dirs, files in os.walk(folder_keypoints):
    for filename in files:
        if not filename.startswith('static') and filename.endswith('.txt'):
            dynamic_file = os.path.join(root, filename)
            number = re.search(r'\d+', filename).group()

            keypoints_data = get_coordinates(dynamic_file)

            # Find corresponding dynamic file in MediaPipePose folder
            for root1, dirs1, files1 in os.walk(folder_keypoints_1):
                for filename1 in files1:
                    number1 = re.search(r'\d+', filename1).group()
                    if not filename1.startswith('static') and filename1.endswith('.txt') and number1 == number:
                        dynamic_file1 = os.path.join(root1, filename1)
                        keypoints_data1 = get_coordinates(dynamic_file1)

                        for frame_number, keypoints in keypoints_data.items():
                            # Right EL, EM
                            shoulder_right = np.array(keypoints[15])
                            elbow_right = np.array(keypoints[16])
                            wrist_right = np.array(keypoints[17])

                            vector1_right = shoulder_right - elbow_right
                            vector1_right_unit = vector1_right / np.linalg.norm(vector1_right)
                            vector2_right = wrist_right - elbow_right
                            vector2_right_unit = vector2_right / np.linalg.norm(vector2_right)

                            normal_vector_right = np.cross(vector1_right, vector2_right) #vector perpendicular to the plane formed by the right shoulder, elbow and wrist, pointing to the left
                            normal_vector_right_unit = normal_vector_right / np.linalg.norm(normal_vector_right)

                            normal_vector1_right = normal_vector_right_unit * ((np.linalg.norm(shoulder_right - elbow_right) * (0.0632 / 2)) / 0.2655) #ratio between half the distance from EM to EL and the distance from shoulder to elbow (midpoint EM-EL)

                            in_elbow_right = elbow_right + normal_vector1_right
                            out_elbow_right = elbow_right - normal_vector1_right

                            keypoints[20] = in_elbow_right.tolist()  # Right EM
                            keypoints[21] = out_elbow_right.tolist()  # Right EL

                            # Left EL, EM
                            shoulder_left = np.array(keypoints[12])
                            elbow_left = np.array(keypoints[13])
                            wrist_left = np.array(keypoints[14])

                            vector1_left = shoulder_left - elbow_left
                            vector1_left_unit = vector1_left / np.linalg.norm(vector1_left)
                            vector2_left = wrist_left - elbow_left
                            vector2_left_unit = vector2_left / np.linalg.norm(vector2_left)
                            normal_vector_left = np.cross(vector1_left, vector2_left)
                            normal_vector_left_unit = normal_vector_left / np.linalg.norm(normal_vector_left)

                            normal_vector1_left = normal_vector_left_unit * ((np.linalg.norm(shoulder_left - elbow_left) * (0.0632 / 2)) / 0.2655)

                            in_elbow_left = elbow_left - normal_vector1_left
                            out_elbow_left = elbow_left + normal_vector1_left

                            keypoints[22] = in_elbow_left.tolist()  # Left EM
                            keypoints[23] = out_elbow_left.tolist()  # Left EL

                        # Calculate RS, US for each frame
                        for frame_number, keypoints in keypoints_data.items():
                            index_right = np.array(keypoints_data1[frame_number][21])
                            pinky_right = np.array(keypoints_data1[frame_number][19])
                            wrist_right = np.array(keypoints[17])
                            elbow_right = np.array(keypoints[16])

                            vector_wrist_1 = index_right - pinky_right
                            vector_wrist_unit_1 = vector_wrist_1 / np.linalg.norm(vector_wrist_1)

                            vector_wrist_right = vector_wrist_unit_1 * ((np.linalg.norm(elbow_right - wrist_right) * (0.0351 / 2)) / 0.2312) #ratio between half the distance from US to RS and the distance from the elbow to the wrist
                            in_wrist_right = wrist_right - vector_wrist_right
                            out_wrist_right = wrist_right + vector_wrist_right

                            keypoints[24] = in_wrist_right.tolist()  # Right US
                            keypoints[25] = out_wrist_right.tolist()  # Right RS

                            index_left = np.array(keypoints_data1[frame_number][20])
                            pinky_left = np.array(keypoints_data1[frame_number][18])
                            wrist_left = np.array(keypoints[14])
                            elbow_left = np.array(keypoints[13])

                            vector_wrist_2 = index_left - pinky_left
                            vector_wrist_unit_2 = vector_wrist_2 / np.linalg.norm(vector_wrist_2)

                            vector_wrist_left = vector_wrist_unit_2 * ((np.linalg.norm(elbow_left - wrist_left) * (0.0351 / 2)) / 0.2312)
                            in_wrist_left = wrist_left - vector_wrist_left
                            out_wrist_left = wrist_left + vector_wrist_left

                            keypoints[26] = in_wrist_left.tolist()  # Left US
                            keypoints[27] = out_wrist_left.tolist()  # Left RS

                        # Calculate Right AC
                        for frame_number, keypoints in keypoints_data.items():
                            IJ = np.array(keypoints[18])
                            C7 = np.array(keypoints[9])
                            T8 = np.array(keypoints[8])
                            PX = np.array(keypoints[19])
                            shoulder_right = np.array(keypoints[15])

                            MidIJC7 = (IJ + C7) / 2
                            MidPXT8 = (PX + T8) / 2

                            Y = (MidIJC7 - MidPXT8) / np.linalg.norm(MidIJC7 - MidPXT8)

                            IJC7 = (C7 - IJ) / np.linalg.norm(C7 - IJ)
                            IJMid = (MidPXT8 - IJ) / np.linalg.norm(MidPXT8 - IJ)

                            Z = np.cross(IJC7, IJMid) / np.linalg.norm(np.cross(IJC7, IJMid))

                            X = np.cross(Y, Z) / np.linalg.norm(np.cross(Y, Z))

                            A = np.column_stack((X, Y, Z))

                            SJC_local = np.dot(A.T, (shoulder_right - IJ).reshape(-1, 1))

                            AC_local = SJC_local - np.array([[12e-3], [-45e-3], [6e-3]])

                            AC_r = IJ.reshape(-1, 1) + np.dot(A, AC_local)

                            keypoints[28] = AC_r.flatten().tolist()  # Right AC

                        # Calculate Left AC
                        for frame_number, keypoints in keypoints_data.items():
                            IJ = np.array(keypoints[18])
                            C7 = np.array(keypoints[9])
                            T8 = np.array(keypoints[8])
                            PX = np.array(keypoints[19])
                            shoulder_left = np.array(keypoints[12])

                            MidIJC7 = (IJ + C7) / 2
                            MidPXT8 = (PX + T8) / 2

                            Y = (MidIJC7 - MidPXT8) / np.linalg.norm(MidIJC7 - MidPXT8)

                            IJC7 = (C7 - IJ) / np.linalg.norm(C7 - IJ)
                            IJMid = (MidPXT8 - IJ) / np.linalg.norm(MidPXT8 - IJ)

                            Z = np.cross(IJC7, IJMid) / np.linalg.norm(np.cross(IJC7, IJMid))

                            X = np.cross(Y, Z) / np.linalg.norm(np.cross(Y, Z))

                            A = np.column_stack((X, Y, Z))

                            SJC_local = np.dot(A.T, (shoulder_left - IJ).reshape(-1, 1))

                            AC_local = SJC_local - np.array([[12e-3], [-45e-3], [-6e-3]])

                            AC_l = IJ.reshape(-1, 1) + np.dot(A, AC_local)

                            keypoints[29] = AC_l.flatten().tolist()  # Left AC

            # Save calculated keypoints in the dynamic .txt file
            with open(dynamic_file, 'w') as f:
                for frame_number, keypoints in keypoints_data.items():
                    f.write(f"Frame {frame_number}:\n")
                    for keypoint_number, coordinates in keypoints.items():
                        f.write(f"Keypoint {keypoint_number}: {coordinates}\n")

            print("New keypoints added to:", dynamic_file)


In [ ]:
folder_keypoints = '/content/coordinates_VideoPose3D'
folder_keypoints_1 = '/content/coordinates_MediaPipePose'

# Calculate EL, EM, RS, US, AC for the static trial
for root, dirs, files in os.walk(folder_keypoints):
    for filename in files:
        if filename.startswith('static') and filename.endswith('.txt'):
            static_file = os.path.join(root, filename)
            number = re.search(r'\d+', filename).group()

            keypoints_data = get_coordinates(static_file)

            # Find corresponding static file in MediaPipePose folder
            for root1, dirs1, files1 in os.walk(folder_keypoints_1):
                for filename1 in files1:
                    number1 = re.search(r'\d+', filename1).group()
                    if filename1.startswith('static') and filename1.endswith('.txt') and number1 == number:
                        static_file1 = os.path.join(root1, filename1)
                        keypoints_data1 = get_coordinates(static_file1)

                        for frame_number, keypoints in keypoints_data.items():
                            # Right EL, EM
                            shoulder_right = np.array(keypoints[15])
                            elbow_right = np.array(keypoints[16])
                            wrist_right = np.array(keypoints[17])

                            vector1_right = shoulder_right - elbow_right
                            vector1_right_unit = vector1_right / np.linalg.norm(vector1_right)
                            vector2_right = wrist_right - elbow_right
                            vector2_right_unit = vector2_right / np.linalg.norm(vector2_right)

                            normal_vector_right = np.cross(vector1_right, vector2_right)
                            normal_vector_right_unit = normal_vector_right / np.linalg.norm(normal_vector_right)

                            normal_vector1_right = normal_vector_right_unit * ((np.linalg.norm(shoulder_right - elbow_right) * (0.0632 / 2)) / 0.2655)

                            in_elbow_right = elbow_right + normal_vector1_right
                            out_elbow_right = elbow_right - normal_vector1_right

                            keypoints[20] = in_elbow_right.tolist()  # Right EM
                            keypoints[21] = out_elbow_right.tolist()  # Right EL

                            # Left EL, EM
                            shoulder_left = np.array(keypoints[12])
                            elbow_left = np.array(keypoints[13])
                            wrist_left = np.array(keypoints[14])

                            vector1_left = shoulder_left - elbow_left
                            vector1_left_unit = vector1_left / np.linalg.norm(vector1_left)
                            vector2_left = wrist_left - elbow_left
                            vector2_left_unit = vector2_left / np.linalg.norm(vector2_left)
                            normal_vector_left = np.cross(vector1_left, vector2_left)
                            normal_vector_left_unit = normal_vector_left / np.linalg.norm(normal_vector_left)

                            normal_vector1_left = normal_vector_left_unit * ((np.linalg.norm(shoulder_left - elbow_left) * (0.0632 / 2)) / 0.2655)

                            in_elbow_left = elbow_left - normal_vector1_left
                            out_elbow_left = elbow_left + normal_vector1_left

                            keypoints[22] = in_elbow_left.tolist()  # Left EM
                            keypoints[23] = out_elbow_left.tolist()  # Left EL

                        # Calculate RS, US for each frame
                        for frame_number, keypoints in keypoints_data.items():
                            index_right = np.array(keypoints_data1[frame_number][21])
                            pinky_right = np.array(keypoints_data1[frame_number][19])
                            wrist_right = np.array(keypoints[17])
                            elbow_right = np.array(keypoints[16])

                            vector_wrist_1 = index_right - pinky_right
                            vector_wrist_unit_1 = vector_wrist_1 / np.linalg.norm(vector_wrist_1)

                            vector_wrist_right = vector_wrist_unit_1 * ((np.linalg.norm(elbow_right - wrist_right) * (0.0351 / 2)) / 0.2312)
                            in_wrist_right = wrist_right - vector_wrist_right
                            out_wrist_right = wrist_right + vector_wrist_right

                            keypoints[24] = in_wrist_right.tolist()  # Right US
                            keypoints[25] = out_wrist_right.tolist()  # Right RS

                            index_left = np.array(keypoints_data1[frame_number][20])
                            pinky_left = np.array(keypoints_data1[frame_number][18])
                            wrist_left = np.array(keypoints[14])
                            elbow_left = np.array(keypoints[13])

                            vector_wrist_2 = index_left - pinky_left
                            vector_wrist_unit_2 = vector_wrist_2 / np.linalg.norm(vector_wrist_2)

                            vector_wrist_left = vector_wrist_unit_2 * ((np.linalg.norm(elbow_left - wrist_left) * (0.0351 / 2)) / 0.2312)
                            in_wrist_left = wrist_left - vector_wrist_left
                            out_wrist_left = wrist_left + vector_wrist_left

                            keypoints[26] = in_wrist_left.tolist()  # Left US
                            keypoints[27] = out_wrist_left.tolist()  # Left RS

                        # Calculate Right AC
                        for frame_number, keypoints in keypoints_data.items():
                            IJ = np.array(keypoints[18])
                            C7 = np.array(keypoints[9])
                            T8 = np.array(keypoints[8])
                            PX = np.array(keypoints[19])
                            shoulder_right = np.array(keypoints[15])

                            MidIJC7 = (IJ + C7) / 2
                            MidPXT8 = (PX + T8) / 2

                            Y = (MidIJC7 - MidPXT8) / np.linalg.norm(MidIJC7 - MidPXT8)

                            IJC7 = (C7 - IJ) / np.linalg.norm(C7 - IJ)
                            IJMid = (MidPXT8 - IJ) / np.linalg.norm(MidPXT8 - IJ)

                            Z = np.cross(IJC7, IJMid) / np.linalg.norm(np.cross(IJC7, IJMid))

                            X = np.cross(Y, Z) / np.linalg.norm(np.cross(Y, Z))

                            A = np.column_stack((X, Y, Z))

                            SJC_local = np.dot(A.T, (shoulder_right - IJ).reshape(-1, 1))

                            AC_local = SJC_local - np.array([[12e-3], [-45e-3], [6e-3]])

                            AC_r = IJ.reshape(-1, 1) + np.dot(A, AC_local)

                            keypoints[28] = AC_r.flatten().tolist()  # Right AC

                        # Calculate Left AC
                        for frame_number, keypoints in keypoints_data.items():
                            IJ = np.array(keypoints[18])
                            C7 = np.array(keypoints[9])
                            T8 = np.array(keypoints[8])
                            PX = np.array(keypoints[19])
                            shoulder_left = np.array(keypoints[12])

                            MidIJC7 = (IJ + C7) / 2
                            MidPXT8 = (PX + T8) / 2

                            Y = (MidIJC7 - MidPXT8) / np.linalg.norm(MidIJC7 - MidPXT8)

                            IJC7 = (C7 - IJ) / np.linalg.norm(C7 - IJ)
                            IJMid = (MidPXT8 - IJ) / np.linalg.norm(MidPXT8 - IJ)

                            Z = np.cross(IJC7, IJMid) / np.linalg.norm(np.cross(IJC7, IJMid))

                            X = np.cross(Y, Z) / np.linalg.norm(np.cross(Y, Z))

                            A = np.column_stack((X, Y, Z))

                            SJC_local = np.dot(A.T, (shoulder_left - IJ).reshape(-1, 1))

                            AC_local = SJC_local - np.array([[12e-3], [-45e-3], [-6e-3]])

                            AC_l = IJ.reshape(-1, 1) + np.dot(A, AC_local)

                            keypoints[29] = AC_l.flatten().tolist()  # Left AC

            # Save calculated keypoints in the static .txt file
            with open(static_file, 'w') as f:
                for frame_number, keypoints in keypoints_data.items():
                    f.write(f"Frame {frame_number}:\n")
                    for keypoint_number, coordinates in keypoints.items():
                        f.write(f"Keypoint {keypoint_number}: {coordinates}\n")

            print("New keypoints added to:", static_file)


**Step 4 -** Plot the skeleton-based model for a given frame of a given input video.

In [ ]:
#plot Hybrid

import math
import plotly.graph_objs as go


#function to calculate the mid point
def calculate_midpoint(point1, point2):
    x0, y0, z0 = point1
    x1, y1, z1 = point2
    midpoint = ((x0 + x1) / 2, (y0 + y1) / 2, (z0 + z1) / 2)
    return midpoint

file_name = input("Enter the directory of the file: ")
frame_VideoPose3D = int(input("Enter the frame number: "))

keypoints_file = file_name
keypoints_data = get_coordinates(keypoints_file)


#Hybrid keypoints + additional estimated keypoints
keypoints_to_name = {
    1: 'pelvis',
    2: 'right hip',
    3: 'right knee',
    4: 'right anke',
    5: 'left hip',
    6: 'left knee',
    7: 'left ankle',
    8: 'T8',
    9: 'C7',
    10: 'nose',
    11: 'head',
    12: 'left shoulder',
    13: 'left elbow',
    14: 'left wrist',
    15: 'right shoulder',
    16: 'right elbow',
    17: 'right wrist',
    18: 'IJ',
    19: 'PX',
    20: 'right EM',
    21: 'right EL',
    22: 'left EM',
    23: 'left EL',
    24: 'right US',
    25: 'right RS',
    26: 'left US',
    27: 'left RS',
    28: 'right AC',
    29: 'left AC'
}


#skeleton for Hybrid keypoints
skeleton_connections = [
    (11, 10), (10, (18,9)),  #head
    ((18, 9), (15, 28)), ((15,28), 16), (16, 17),  #right arm
    ((18, 9), (12, 29)), ((12,29), 13), (13, 14),  #left arm
    ((18, 9), (19, 8)), ((19,8), 1),  #trunk
    (1, 2), (2, 3), (3, 4),  #right leg
    (1, 5), (5, 6), (6, 7)  #left leg
]


#Hybrid referential system

origin1 = np.array(keypoints_data[1][18]) #origin -  IJ
mid_PX_T81 = (np.array(keypoints_data[1][19]) + np.array(keypoints_data[1][8]))/2 #midpoint of PX T8
mid_IJ_C71 = (np.array(keypoints_data[1][18]) + np.array(keypoints_data[1][9]))/2 #midpoint of IJ C7
y1= mid_IJ_C71 - mid_PX_T81
y_axis1 = y1/ np.linalg.norm(y1)
vector_1 = np.array(keypoints_data[1][18]) - np.array(keypoints_data[1][9]) #vector C7 - IJ
vector_2 = mid_PX_T81 - np.array(keypoints_data[1][9]) #vector C7 - midpoint of PX T8
vector_1_unit=vector_1/ np.linalg.norm(vector_1)
vector_2_unit=vector_2/ np.linalg.norm(vector_2)
z_axis1= -(np.cross(vector_1_unit, vector_2_unit)) / np.linalg.norm(np.cross(vector_1_unit, vector_2_unit)) #perpendicular vector to the plane formed by IJ, C7 e mid PX-T8
x_axis1 = (np.cross(y_axis1, z_axis1)) / np.linalg.norm(np.cross(y_axis1, z_axis1))
basis1 = np.array([[x_axis1[0], y_axis1[0], z_axis1[0]],
                      [x_axis1[1], y_axis1[1], z_axis1[1]],
                      [x_axis1[2], y_axis1[2], z_axis1[2]]])

#transform the coordinates to the defined reference
new_keypoints = {}
for keypoint_number, coordinates in keypoints_data[frame_VideoPose3D].items():
    pointP1 = np.array(coordinates)
    new_point1 = (basis1.T).dot(pointP1 - origin1)
    new_keypoints[keypoint_number] = new_point1


desired_keypoints = [8, 9, 18, 19, 20, 21, 24, 25,28]
desired_keypoints1 =[22,23,26,27,29]
data_plot = []

for keypoint_number, coordinates in new_keypoints.items():
    x, y, z = coordinates
    keypoint_name = keypoints_to_name.get(keypoint_number)
    if keypoint_number == 8:
        text = 'T8'
    elif keypoint_number == 9:
        text = 'C7'
    elif keypoint_number == 18:
        text = 'IJ'
    elif keypoint_number == 19:
        text = 'PX'
    elif keypoint_number == 20:
        text = 'EM'
    elif keypoint_number == 21:
        text = 'EL'
    elif keypoint_number == 24:
        text = 'US'
    elif keypoint_number == 25:
        text = 'RS'
    elif keypoint_number == 28:
        text = 'AC'
    else:
        text = None

    trace = go.Scatter3d(
        x=[x],
        y=[y],
        z=[z],
        mode='markers+text' if text else 'markers',
        marker=dict(
            size=5,
            color = 'orange' if (keypoint_number in desired_keypoints or keypoint_number in desired_keypoints1) else 'blue',
            symbol='circle'
        ),
        text=[text],
        textposition='bottom center',
        textfont=dict(size=10),
        name=f'{keypoint_number} - {keypoint_name}'
    )
    data_plot.append(trace)

#connect the different keypoints
for connection in skeleton_connections:
    if isinstance(connection[0], tuple) and isinstance(connection[1], tuple): #when the first and the second are tupples
        mid1 = calculate_midpoint(new_keypoints[connection[0][0]], new_keypoints[connection[0][1]])
        mid2 = calculate_midpoint(new_keypoints[connection[1][0]], new_keypoints[connection[1][1]])
        trace = go.Scatter3d(
            x=[mid1[0], mid2[0]],
            y=[mid1[1], mid2[1]],
            z=[mid1[2], mid2[2]],
            mode='lines',
            line=dict(
                color='blue',
                width=2
            ),
            showlegend=False
        )
    elif isinstance(connection[0], tuple): #only the first is a tupple
        mid = calculate_midpoint(new_keypoints[connection[0][0]], new_keypoints[connection[0][1]])
        point = new_keypoints[connection[1]]
        trace = go.Scatter3d(
            x=[mid[0], point[0]],
            y=[mid[1], point[1]],
            z=[mid[2], point[2]],
            mode='lines',
            line=dict(
                color='blue',
                width=2
            ),
            showlegend=False
        )
    elif isinstance(connection[1], tuple): #only the second is a tupple
        point = new_keypoints[connection[0]]
        mid = calculate_midpoint(new_keypoints[connection[1][0]], new_keypoints[connection[1][1]])
        trace = go.Scatter3d(
            x=[point[0], mid[0]],
            y=[point[1], mid[1]],
            z=[point[2], mid[2]],
            mode='lines',
            line=dict(
                color='blue',
                width=2
            ),
            showlegend=False
        )
    else: #only one keypoint in each side
        point1 = new_keypoints[connection[0]]
        point2 = new_keypoints[connection[1]]
        trace = go.Scatter3d(
            x=[point1[0], point2[0]],
            y=[point1[1], point2[1]],
            z=[point1[2], point2[2]],
            mode='lines',
            line=dict(
                color='blue',
                width=2
            ),
            showlegend=False
        )
    data_plot.append(trace)

#scale of the axis
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X', range=[-1.2, 1.2]),
        yaxis=dict(title='Y', range=[-1.2, 1.2]),
        zaxis=dict(title='Z', range=[-1.2, 1.2]),
        aspectmode='cube',
        dragmode='orbit',
        uirevision=True
    ),
    margin=dict(t=30, r=0, l=20, b=10)
)

fig = go.Figure(data=data_plot, layout=layout)


#to select the angle of view
angle_y = 10

angle_rad = angle_y * (3.14159 / 180)

camera_y_rotation = dict(
    eye=dict(
        x=0.7 * math.cos(angle_rad),
        y=0,
        z=0.7 * math.sin(angle_rad)
    ),
    up=dict(x=0, y=1, z=0),
    center=dict(x=0, y=0, z=0)
)

fig.update_layout(scene_camera=camera_y_rotation)